In [1]:
import jupyter_black

jupyter_black.load()  # Python linter

# Automatically provide the LLM an MCP Tool (during runtime)

## Prerequisites

We will use the `pydantic-ai-slim[mcp]` package for interacting with the MCP Servers 

- Install it with: `pip install baml-agents[pydantic-mcp]`

## Let's get started

First, let's create the BAML file

In [2]:
%cat ../baml_src/02_use_tool.baml

class ToolCall {
    @@dynamic
}

function UseTool(goal: string, output_format_prefix: string) -> ToolCall {
    client Default
    prompt #"
        Solve: {{ goal }}

        {{ ctx.output_format(prefix=output_format_prefix, or_splitter=" OR ")}}
    "#
}


Then, let's add the MCP Server config

In [3]:
from pydantic_ai.mcp import MCPServerStdio

from baml_agents.deprecated.pydantic_mcp import McpServers
from baml_agents.utils import get_payload
from baml_client.async_client import b
from baml_client.type_builder import TypeBuilder

server_with_calculator_tool = McpServers(
    [
        MCPServerStdio(
            command="python",
            args=["-m", "mcp-server-calculator"],
        ),
    ],
)

Automatically fill the LLM prompt with the available MCP Tools from the MCP Server

In [4]:
async with server_with_calculator_tool as server:
    # Add MCP tools to the BAML structured output schema
    tb, tool_runner, p = await server.build_tool_types(
        tb := TypeBuilder(),
        output_class=tb.ToolCall,
        tools=await server.list_tools(),
    )

    # View prompt
    goal = "Multiply all numbers between 10 and 15"
    request = await b.request.UseTool(goal, p, baml_options={"tb": tb})
    print(get_payload(request))

    # LLM chooses the tools
    result = await b.UseTool(goal, p, baml_options={"tb": tb})

    # Run the tools
    tool_results = await tool_runner.run(result)

[system]
Solve: Multiply all numbers between 10 and 15

What are the next steps?

Answer in JSON format with one or multiple of the following intents:

{
  intents: [
    {
      // Calculates/evaluates the given expression.
      intent: "calculate",
      expression: string,
    }
  ],
}

Let's see the LLM-chosen arguments before calling the MCP Tool

In [5]:
result

ToolCall(intents=[{'intent': 'calculate', 'expression': '11 * 12 * 13 * 14'}])

Finally, let's call the MCP Tool and show the result

In [6]:
tool_results

[CallToolResult(meta=None, content=[TextContent(type='text', text='24024', annotations=None)], isError=False)]